**Nom Etudiant : Diallo**  

**Prenom Etudiant:Bineta**  

**Classe :ING2GIT** 

**Date limite de dépot : 31/08/2022**


# Travail à Faire:
Télécharger le Datasets sur le lien Drive :  https://drive.google.com/file/d/1tuV0lyb2uWvOInodVTohpdCiAV5FrBX-/view?usp=sharing 

Repondre les questions ci-dessous avec le maximum de precisions et de détails.   
Remplir `FILL_IN` avec les methodes qui correspondent à la réponse adéquate


### Revenus des achats
1. Extraire les revenus d'achat pour chaque événement
2. Filtrer les événements dont le revenu n'est pas nul
3. Vérifiez quels sont les types d'événements qui génèrent des revenus
4. Supprimez la colonne inutile

### Revenus par Traffic
Obtenir les 3 sources de trafic générant le revenu total le plus élevé.
5. Revenus cumulés par source de trafic
7. Obtenir les 3 principales sources de trafic par revenu total
6. Nettoyer les colonnes de revenus pour avoir deux décimales


##### Methods
- DataFrame: `select`, `drop`, `withColumn`, `filter`, `dropDuplicates`,  `groupBy`, `sort`, `limit`
- Column: `isNotNull`, `alias`, `desc`, `cast`, `operators`

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Session de Rattrapage Big Data").master("local").getOrCreate()
sc=spark.sparkContext
spark

In [3]:
eventsPath = "events.json"

In [5]:
eventsDF = spark.read.option("header","true").option("inteferSchema","true").option("delimiter",",").json(eventsPath)
eventsDF.show(10)

+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|
+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+
|  macOS|{null, null, null}|  warranty|        1593878899217692|1593878946592107|      {Montrose, MI}|                  []|        google|          1593878899217692|UA000000107379500|
|Windows|{null, null, null}|     press|        1593876662175340|1593877011756535|   {Northampton, MA}|                  []|        google|          1593876662175340|UA000000107359357|
|  macOS|{null, null, null}|  add_item|        1593878792892652|1593878815459100

### 1. Extraire les revenus d'achat pour chaque événement
Ajouter une nouvelle colonne **`revenue`** en faisant l'extration de **`ecommerce.purchase_revenue_in_usd`**

In [6]:
revenu=eventsDF.select('ecommerce.purchase_revenue_in_usd')
revenu.show(10)

+-----------------------+
|purchase_revenue_in_usd|
+-----------------------+
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
|                   null|
+-----------------------+
only showing top 10 rows



In [7]:
from pyspark.sql.functions import col
revenueDF = eventsDF.withColumn("revenue",col("ecommerce.purchase_revenue_in_usd"))
revenueDF.show(10)

+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
| device|         ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+-------+------------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|  macOS|{null, null, null}|  warranty|        1593878899217692|1593878946592107|      {Montrose, MI}|                  []|        google|          1593878899217692|UA000000107379500|   null|
|Windows|{null, null, null}|     press|        1593876662175340|1593877011756535|   {Northampton, MA}|                  []|        google|          1593876662175340|UA000000107359357|   null|
|  macOS|{null, null, null}|  add_item| 

### 2. Filtrer les événements dont le revenu n'est pas null

In [10]:
purchasesDF = revenueDF.filter("revenue IS NOT NULL")
purchasesDF.show(10)

+---------+--------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|   device|     ecommerce|event_name|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+---------+--------------+----------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|    Linux|{1195.0, 1, 1}|  finalize|        1593878893766134|1593878897648871|       {Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
|      iOS|{1045.0, 1, 1}|  finalize|        1593878485345763|1593878487460247|       {Detroit, MI}|[{null, M_STAN_Q,...|      facebook|          1593877230282722|UA000000107364432| 1045.0|
|  Android| {595.0, 1, 1}|  finalize|        15938

### 3. Vérifiez quels sont les types d'événements qui génèrent des revenus
Trouvez des valeurs **`event_name`** uniques dans **`purchasesDF`**. Il y a deux façons de faire :
- Sélectionnez "event_name" et recupérer les enregistrements distincts
- Supprimez les enregistrements en double en fonction de "event_name" uniquement

In [12]:
distinctDF = purchasesDF.select('event_name').distinct()
distinctDF.show()

+----------+
|event_name|
+----------+
|  finalize|
+----------+



In [13]:
purchasesDF.dropDuplicates(['event_name']).show()

+------+--------------+----------+------------------------+----------------+-------------+--------------------+--------------+--------------------------+-----------------+-------+
|device|     ecommerce|event_name|event_previous_timestamp| event_timestamp|          geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+------+--------------+----------+------------------------+----------------+-------------+--------------------+--------------+--------------------------+-----------------+-------+
| Linux|{1195.0, 1, 1}|  finalize|        1593878893766134|1593878897648871|{Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
+------+--------------+----------+------------------------+----------------+-------------+--------------------+--------------+--------------------------+-----------------+-------+



### 4. Supprimez la colonne inutile
Puisqu'il n'y a qu'un seul type d'événement, supprimez **`event_name`** de **`purchasesDF`**.

In [16]:
cleanDF = purchasesDF.drop('event_name')
cleanDF.show(10)

+---------+--------------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|   device|     ecommerce|event_previous_timestamp| event_timestamp|                 geo|               items|traffic_source|user_first_touch_timestamp|          user_id|revenue|
+---------+--------------+------------------------+----------------+--------------------+--------------------+--------------+--------------------------+-----------------+-------+
|    Linux|{1195.0, 1, 1}|        1593878893766134|1593878897648871|       {Shawnee, KS}|[{null, M_STAN_K,...|        google|          1593876996316576|UA000000107362263| 1195.0|
|      iOS|{1045.0, 1, 1}|        1593878485345763|1593878487460247|       {Detroit, MI}|[{null, M_STAN_Q,...|      facebook|          1593877230282722|UA000000107364432| 1045.0|
|  Android| {595.0, 1, 1}|        1593877930076602|1593878966392505|  {East Chicago, IN}|[{null, M_STAN_T

### 5. Revenus cumulés par source de trafic
- Grouper par **`traffic_source`**
- Obtenir la somme de **`revenue`** comme **`total_rev`**
- Obtenir la moyenne de **`revenue`** comme **`avg_rev`**

N'oubliez pas d'importer toutes les fonctions intégrées nécessaires.

In [28]:
trafficDF1 = cleanDF.groupBy("traffic_source").sum("revenue")
trafficDF1=trafficDF1.withColumnRenamed("sum(revenue)","total_rev")
trafficDF1.show()

+--------------+---------+
|traffic_source|total_rev|
+--------------+---------+
|     instagram|   8160.0|
|        direct|   9129.0|
|       youtube|   3040.0|
|         email|  36935.4|
|      facebook|  12952.0|
|        google|  28936.0|
+--------------+---------+



In [29]:
trafficDF2=cleanDF.groupBy("traffic_source").avg("revenue")
trafficDF2=trafficDF2.withColumnRenamed("avg(revenue)","avg_rev")
trafficDF2.show()

+--------------+-----------------+
|traffic_source|          avg_rev|
+--------------+-----------------+
|     instagram|           1020.0|
|        direct|         1141.125|
|       youtube|           1520.0|
|         email|998.2540540540541|
|      facebook|996.3076923076923|
|        google|964.5333333333333|
+--------------+-----------------+



### 2. Obtenir les trois principales sources de trafic par revenu total
- Trier par **`total_rev`** dans l'ordre décroissant
- Se limiter aux trois premières lignes

In [30]:
topTrafficDF = trafficDF1.sort(['total_rev'],ascending = False)
topTrafficDF.show(3)

+--------------+---------+
|traffic_source|total_rev|
+--------------+---------+
|         email|  36935.4|
|        google|  28936.0|
|      facebook|  12952.0|
+--------------+---------+
only showing top 3 rows



In [31]:
topTrafficDF2 = trafficDF2.sort(['avg_rev'],ascending = False)
topTrafficDF2.show(3)

+--------------+--------+
|traffic_source| avg_rev|
+--------------+--------+
|       youtube|  1520.0|
|        direct|1141.125|
|     instagram|  1020.0|
+--------------+--------+
only showing top 3 rows



### 3. Limitez les colonnes de revenus à deux décimales pointés
- Modifier les colonnes **`avg_rev`** et **`total_rev`** pour les convertir en des nombres avec deux décimales pointés

In [26]:
finalDF1 = topTrafficDF.withColumn("total_rev", col("total_rev").cast("Decimal(10,2)"))
finalDF1.show()

+--------------+---------+
|traffic_source|total_rev|
+--------------+---------+
|         email| 36935.40|
|        google| 28936.00|
|      facebook| 12952.00|
|        direct|  9129.00|
|     instagram|  8160.00|
|       youtube|  3040.00|
+--------------+---------+



In [32]:
finalDF2 = topTrafficDF2.withColumn("avg_rev", col("avg_rev").cast("Decimal(10,2)"))
finalDF2.show()

+--------------+-------+
|traffic_source|avg_rev|
+--------------+-------+
|       youtube|1520.00|
|        direct|1141.13|
|     instagram|1020.00|
|         email| 998.25|
|      facebook| 996.31|
|        google| 964.53|
+--------------+-------+

